# Step 4: Create models with SD

In [1]:
import pandas as pd 
import os 
import sys 
import pickle


from sklearn.metrics import (classification_report, 
                             roc_auc_score, 
                             matthews_corrcoef,
                             cohen_kappa_score)
sys.path.append('../src')
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   run_pycaret_setup, 
                   translate_model_name,
                   get_synthetic_filepaths_from_original_data_id)

from mlflow_manager import MLFlowManager

# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

In [2]:
settings = dataset_settings[0]

original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}")

synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

sd_filename  = synthetic_datasets[0]

sd_id = os.path.splitext(sd_filename)[0]
sd_path = folders['sd_dir']+sd_filename

s = run_pycaret_setup(sd_path, settings['setup_param'])

ml_model = 'lr'

display(f"Creating {ml_model}")
model = s.create_model(ml_model)
display(f"Tuning the model")
tuned_model = s.tune_model(model, **config['clf']['tuning_param']) 

# get validation results
val_df = s.pull()
val_score = {}
val_score['val_accuracy'] = val_df['Accuracy']['Mean']
val_score['val_f1-score'] = val_df['F1']['Mean']

display(f"Predict on the original data")
holdout_df = s.predict_model(estimator=tuned_model, data=original_data.drop(columns=['Outcome']))

# test the model on the holdout-data
#y_pred = tuned_model.predict(x_test_transformed)
#metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
display(holdout_df)

'Creating lr'

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:12:06
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7258,0.7677,0.6562,0.7778,0.7119,0.4539,0.4599
1,0.7419,0.8188,0.7188,0.7667,0.7419,0.4844,0.4854
2,0.7581,0.7417,0.8125,0.7429,0.7761,0.5141,0.5166
3,0.6613,0.7302,0.6875,0.6667,0.6769,0.3212,0.3213
4,0.6885,0.7575,0.7188,0.6970,0.7077,0.3745,0.3747
5,0.6885,0.7834,0.7500,0.6857,0.7164,0.3725,0.3743
6,0.6393,0.7931,0.7500,0.6316,0.6857,0.2699,0.2754
7,0.6885,0.8473,0.6774,0.7000,0.6885,0.3772,0.3774
8,0.6393,0.6903,0.5806,0.6667,0.6207,0.2800,0.2825


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

'Tuning the model'

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:12:30
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


Fitting 10 folds for each of 2 candidates, totalling 20 fits


'Predict on the original data'

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,prediction_label,prediction_score
0,6,148,72,35,0,33.599998,0.627,50,1,0.5937
1,1,85,66,29,0,26.600000,0.351,31,1,0.7490
2,8,183,64,0,0,23.299999,0.672,32,0,0.6384
3,1,89,66,23,94,28.100000,0.167,21,1,0.8755
4,0,137,40,35,168,43.099998,2.288,33,1,0.8989
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.900002,0.171,63,1,0.8499
764,2,122,70,27,0,36.799999,0.340,27,1,0.7435
765,5,121,72,23,112,26.200001,0.245,30,1,0.8037
766,1,126,60,0,0,30.100000,0.349,47,1,0.5116


In [3]:
y_true = original_data[settings['meta']['target']]
x_test = original_data.drop(columns=[settings['meta']['target']])
y_pred = s.predict_model(estimator=tuned_model, data=x_test)
y_pred_score = y_pred['prediction_score']
y_pred = y_pred['prediction_label']

metrics =  classification_report(y_true=y_true, y_pred=y_pred, output_dict=True, digits=4)
display(metrics)

{'0': {'precision': 0.6028708133971292,
  'recall': 0.252,
  'f1-score': 0.3554301833568406,
  'support': 500},
 '1': {'precision': 0.33094812164579607,
  'recall': 0.6902985074626866,
  'f1-score': 0.44740024183796856,
  'support': 268},
 'accuracy': 0.4049479166666667,
 'macro avg': {'precision': 0.4669094675214627,
  'recall': 0.4711492537313433,
  'f1-score': 0.4014152125974046,
  'support': 768},
 'weighted avg': {'precision': 0.5079811240880703,
  'recall': 0.4049479166666667,
  'f1-score': 0.3875239016809842,
  'support': 768}}

In [4]:
thi = pd.DataFrame.from_dict(metrics).transpose()
display(thi)

from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_true=y_true, y_score=y_pred_score, multi_class='raise')
display(auc)

,precision,recall,f1-score,support
0,0.602871,0.252000,0.355430,500.000000
1,0.330948,0.690299,0.447400,268.000000
accuracy,0.404948,0.404948,0.404948,0.404948
macro avg,0.466909,0.471149,0.401415,768.000000
weighted avg,0.507981,0.404948,0.387524,768.000000


0.5210671641791045

In [2]:
# TODO: consider testing, and consider extracting hyperparameters, consider how to summarize
for settings in dataset_settings:
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False
    
    # Get experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"
    mlflow = MLFlowManager(experiment_name)
    
    # no need to extract from mlflow the test_data, it is no longer relevant
    #test_data = mlflow.get_test_holdout_data()  
    original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}")
    y_true = original_data[settings['meta']['target']]
    x_test = original_data.drop(columns=[settings['meta']['target']])

    logg_tags = {'Dataset Type': 'synthetic'}
    mlflow.start_run('Synthetic data models', tags=logg_tags)
    
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

    for sd_filename in synthetic_datasets:
        
        sd_id = os.path.splitext(sd_filename)[0]
        sd_path = folders['sd_dir']+sd_filename
        
        s = run_pycaret_setup(sd_path, settings['setup_param'])
        USI = s.get_config('USI')
        
        # Add custom tags to the logg, defining dataset type, and Id
        logg_tags = {
            'Dataset Type': 'synthetic',
            'Dataset ID': sd_id,
            'USI': USI
        }
        run_name = f"{sd_id}-models"
        mlflow.start_run(run_name, tags=logg_tags, nested=True)
        
        
        for ml_model in config['clf']['ml_models']:
            #start log run
            model_name = f"{sd_id}-{translate_model_name(ml_model)}"
            mlflow.start_run(model_name, tags=logg_tags, nested=True)
            
            # create & tune model
            model = s.create_model(ml_model)
            tuned_model = s.tune_model(model, **config['clf']['tuning_param']) 
        
            # get validation results
            val_df = s.pull()
            val_score = {}
            val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
            val_score['val_F1-score'] = val_df['F1']['Mean']


            pred_model = s.predict_model(estimator=tuned_model, data=x_test)
            y_pred = pred_model['prediction_label']

            metrics =  classification_report(y_true=y_true, y_pred=y_pred, output_dict=True, digits=4)
            test_score = pd.DataFrame.from_dict(metrics).transpose()
            
            test_metrics = {
                "Accuracy": metrics['accuracy'],
                "F1-score": metrics['macro avg']['f1-score'],
                "MCC": matthews_corrcoef(y_true=y_true, y_pred=y_pred),
                "Kappa": cohen_kappa_score(y1=y_true, y2=y_pred)
            }

            # If there is a prediction_score in the from predict_model
            if 'prediction_score' in pred_model.columns:
                y_pred_score = pred_model['prediction_score']
                # compute AUC
                if original_data[settings['meta']['target']].nunique() > 2:
                    m_class = 'ovo'
                else:
                    m_class = 'raise'
                test_metrics['AUC'] = roc_auc_score(y_true=y_true, y_score=y_pred_score, multi_class=m_class)


            # log parameters     
            mlflow.log_params(tuned_model.get_params())
            # log performance
            mlflow.log_metric_report(test_metrics)
            mlflow.log_metrics(val_score)
            mlflow.log_score_report_to_html(val_df, "Validation")
            mlflow.log_score_report_to_html(test_score, "Test_score")
            # log model
            mlflow.log_model(model=tuned_model)
            # end logging run for the model
            mlflow.end_run()
            
        # end logging run for SD_id
        mlflow.end_run()

    # end logging for the sd_meta synthetic dataset
    mlflow.end_run()          

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:13:39
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6935,0.7385,0.6562,0.7241,0.6885,0.3884,0.3902
1,0.7097,0.7292,0.7188,0.7188,0.7188,0.4188,0.4188
2,0.6129,0.6990,0.5938,0.6333,0.6129,0.2266,0.2271
3,0.6129,0.7125,0.5938,0.6333,0.6129,0.2266,0.2271
4,0.7049,0.7570,0.8387,0.6667,0.7429,0.4071,0.4220
5,0.6393,0.7097,0.6452,0.6452,0.6452,0.2785,0.2785
6,0.7049,0.7441,0.6129,0.7600,0.6786,0.4116,0.4197
7,0.6557,0.7371,0.6562,0.6774,0.6667,0.3109,0.3111
8,0.7049,0.7974,0.7188,0.7188,0.7188,0.4084,0.4084


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6935,0.7385,0.6562,0.7241,0.6885,0.3884,0.3902
1,0.7097,0.7292,0.7188,0.7188,0.7188,0.4188,0.4188
2,0.6129,0.6990,0.5938,0.6333,0.6129,0.2266,0.2271
3,0.6129,0.7125,0.5938,0.6333,0.6129,0.2266,0.2271
4,0.7049,0.7570,0.8387,0.6667,0.7429,0.4071,0.4220
5,0.6393,0.7097,0.6452,0.6452,0.6452,0.2785,0.2785
6,0.7049,0.7441,0.6129,0.7600,0.6786,0.4116,0.4197
7,0.6557,0.7371,0.6562,0.6774,0.6667,0.3109,0.3111
8,0.7049,0.7974,0.7188,0.7188,0.7188,0.4084,0.4084


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:14:02
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6935,0.7385,0.6562,0.7241,0.6885,0.3884,0.3902
1,0.7097,0.7292,0.7188,0.7188,0.7188,0.4188,0.4188
2,0.6129,0.7000,0.5938,0.6333,0.6129,0.2266,0.2271
3,0.6129,0.7125,0.5938,0.6333,0.6129,0.2266,0.2271
4,0.7049,0.7570,0.8387,0.6667,0.7429,0.4071,0.4220
5,0.6393,0.7097,0.6452,0.6452,0.6452,0.2785,0.2785
6,0.7049,0.7441,0.6129,0.7600,0.6786,0.4116,0.4197
7,0.6557,0.7371,0.6562,0.6774,0.6667,0.3109,0.3111
8,0.7049,0.7963,0.7188,0.7188,0.7188,0.4084,0.4084


...

,Description,Value
0,Session id,5133
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(614, 9)"
6,Transformed test set shape,"(154, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:20:32
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8871,0.9458,0.8750,0.9032,0.8889,0.7742,0.7746
1,0.8871,0.9531,0.8438,0.9310,0.8852,0.7747,0.7783
2,0.8871,0.9365,0.9062,0.8788,0.8923,0.7737,0.7741
3,0.8065,0.9115,0.8438,0.7941,0.8182,0.6117,0.6130
4,0.9016,0.9753,0.9677,0.8571,0.9091,0.8028,0.8098
5,0.8689,0.9430,0.9032,0.8485,0.8750,0.7374,0.7389
6,0.9344,0.9752,0.9375,0.9375,0.9375,0.8685,0.8685
7,0.8852,0.9580,0.9688,0.8378,0.8986,0.7681,0.7788
8,0.9016,0.9677,0.8750,0.9333,0.9032,0.8034,0.8052


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [ ]:
df=s.predict_model(estimator=tuned_model, data=x_test)
display(df)
if 'prediction_score' in df.columns:
    print("yes")
if 'prediction_label' in df.columns:
    print("yesyes")


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,prediction_label
0,6,148,72,35,0,33.599998,0.627,50,1
1,1,85,66,29,0,26.600000,0.351,31,1
2,8,183,64,0,0,23.299999,0.672,32,0
3,1,89,66,23,94,28.100000,0.167,21,1
4,0,137,40,35,168,43.099998,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.900002,0.171,63,1
764,2,122,70,27,0,36.799999,0.340,27,1
765,5,121,72,23,112,26.200001,0.245,30,1
766,1,126,60,0,0,30.100000,0.349,47,1


yesyes
